In [12]:
import sys
sys.path.append('/home/zhw272/programming/pipeline_utility/in_development/Will')
sys.path.append('/home/zhw272/programming/pipeline_utility/src')
from cell_extractor.CellDetectorTrainer import CellDetectorTrainer
from cell_extractor.DetectorMetrics import DetectorMetricsDK55
import pandas as pd
import os

In [6]:
trainer = CellDetectorTrainer('DK55',round=2,segmentation_threshold=2300)
trainer.list_detectors()

{'eta': 0.3, 'objective': 'binary:logistic', 'nthread': 7}


['detector_round_2_threshold_2300.pkl',
 'detector_round_2_threshold_2200.pkl',
 'detector_round_2_threshold_2100.pkl',
 'detector_round_2.pkl',
 'detector_round_2_threshold_2000.pkl',
 'detector_round_1.pkl',
 'detector_round_2_threshold_2700.pkl']

In [7]:
all_detections = []
for threshold in [2000,2100,2200,2300,2700]:
    trainer = CellDetectorTrainer('DK55',round=2,segmentation_threshold=threshold)
    detections = trainer.load_detections()
    detections = detections[['row','col','section','predictions']]
    detections['name'] = str(threshold)
    all_detections.append(detections)
all_detections = pd.concat(all_detections)

{'eta': 0.3, 'objective': 'binary:logistic', 'nthread': 7}
{'eta': 0.3, 'objective': 'binary:logistic', 'nthread': 7}
{'eta': 0.3, 'objective': 'binary:logistic', 'nthread': 7}
{'eta': 0.3, 'objective': 'binary:logistic', 'nthread': 7}
{'eta': 0.3, 'objective': 'binary:logistic', 'nthread': 7}


In [8]:
sures = all_detections[all_detections.predictions==2]
unsures = all_detections[all_detections.predictions==0]

In [12]:
util = AnnotationProximityTool()
util.set_annotations_to_compare(unsures)
util.find_equivalent_points()
pair_id = [pair[0] for pair in util.pairs.values()]
unsure_cells = util.annotations_to_compare.iloc[pair_id]

calculating distance matrix
finding points that are close to each other
grouping and labeling points that are close to each other
before removing duplicates 1486
after removing duplicates 553


In [14]:
sures = sures.drop('predictions',axis=1)
unsures = unsures.drop('predictions',axis=1)
sures.columns = ['x','y','section','name']
unsures.columns = ['x','y','section','name']

In [20]:
sure_cells.head()

,x,y,section,name
0,18851,37633,132,2000
1,17423,38133,145,2000
2,17638,37839,145,2000
3,17708,37725,145,2000
4,17794,37634,145,2000


In [30]:
mec = DetectorMetricsDK55('DK55',round =1)
qc_annotation_input_path = '/home/zhw272/programming/pipeline_utility/in_development/yoav/marked_cell_detector/data2/'
sure_cells = sure_cells.drop('prediction',axis=1)
unsure_cells = unsure_cells.drop('prediction',axis=1)
sure_cells.to_csv(qc_annotation_input_path+'multi-threshold_cell_detection(round3)sure.csv',header=False, index=False)
unsure_cells.to_csv(qc_annotation_input_path+'multi-threshold_cell_detection(round3)unsure.csv',header=False, index=False)

In [32]:
mec = DetectorMetricsDK55('DK55',round =1,sure_file_name = 'multi-threshold_cell_detection(round3)sure.csv',unsure_file_name='multi-threshold_cell_detection(round3)unsure.csv')
mec.qc_annotation_input_path = '/home/zhw272/programming/pipeline_utility/in_development/yoav/marked_cell_detector/data2/'

In [33]:
a = mec.load_human_qc()
b = mec.load_machine_detection()

In [36]:
mec.load_annotations_to_compare()
mec.calculate_qualification()

calculating distance matrix
finding points that are close to each other
grouping and labeling points that are close to each other
before removing duplicates 16462
after removing duplicates 16035
computer missed, human detected
705
computer sure, human negative
0
computer  UNsure, human negative
0
computer UNsure, human positive
0
Total computer UNsure
552
computer UNsure, human unmarked
362
computer sure, human unmarked
12548
More than 2 labels
6
Computer Detected, Human Missed
12548
total train
1905
Human mind change
32
original training set after mind change
1873


In [24]:
mec.annotations_to_compare

,0,1,2,name,3,4
0,36418.769231,19700.2,124,manual_train,NaN,NaN
1,36386.153846,19514.8,124,manual_train,NaN,NaN
2,41504.615385,14066.1,128,manual_train,NaN,NaN
3,41209.846154,14259.3,136,manual_train,NaN,NaN
4,41190.769231,14322.1,136,manual_train,NaN,NaN
...,...,...,...,...,...,...
549,12391.000000,14217,14082,computer_unsure,201,0
550,12449.000000,20817,40885,computer_unsure,201,0
551,12564.000000,11163,30752,computer_unsure,175,0
552,12569.000000,17097,37947,computer_unsure,175,0


In [ ]:
"util = AnnotationProximityTool()\n",
"util.set_annotations_to_compare(unsures)\n",
"util.find_equivalent_points()\n",
"pair_id = [pair[0] for pair in util.pairs.values()]\n",
"unsure_cells = util.annotations_to_compare.iloc[pair_id]"